In [1]:
from vlhf.hugging_face import HuggingFace
from vlhf.visual_layer import VisualLayer
import os
from dotenv import load_dotenv

# Authentication
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
VL_USER_ID = os.getenv("VL_USER_ID")
VL_ENV = os.getenv("VL_ENV")

hf = HuggingFace(HF_TOKEN)
vl = VisualLayer(VL_USER_ID, VL_ENV)

2024-08-07 16:59:32.449 | INFO     | vlhf.hugging_face:__init__:64 - Hugging Face session created
2024-08-07 16:59:32.450 | INFO     | vlhf.visual_layer:__init__:20 - Visual Layer session created


In [ ]:
# hf.download_dataset("keremberke/german-traffic-sign-detection", name="full", image_key="image", bbox_key="objects", bbox_label_names=['animals', 'construction', 'cycles crossing', 'danger', 'no entry', 'pedestrian crossing', 'school crossing', 'snow', 'stop', 'bend', 'bend left', 'bend right', 'give way', 'go left', 'go left or straight', 'go right', 'go right or straight', 'go straight', 'keep left', 'keep right', 'no overtaking', 'no overtaking -trucks-', 'no traffic both ways', 'no trucks', 'priority at next intersection', 'priority road', 'restriction ends', 'restriction ends -overtaking -trucks--', 'restriction ends -overtaking-', 'restriction ends 80', 'road narrows', 'roundabout', 'slippery road', 'speed limit 100', 'speed limit 120', 'speed limit 20', 'speed limit 30', 'speed limit 50', 'speed limit 60', 'speed limit 70', 'speed limit 80', 'traffic signal', 'uneven road'])

In [ ]:
# hf.download_dataset("keremberke/csgo-object-detection", name="full", 
#                     image_key="image", 
#                     bbox_key="objects", 
#                     bbox_label_names=['ct', 'cthead', 't', 'thead']
# )

In [2]:
hf.download_dataset("rishitdagli/cppe-5", 
                    image_key="image", 
                    bbox_key="objects", 
                    bbox_label_names=["coverall", "face_shield", "gloves", "goggles", "mask"])

2024-08-07 17:00:57.080 | INFO     | vlhf.hugging_face:download_dataset:110 - Downloading dataset rishitdagli/cppe-5 and saving to local path rishitdagli/cppe-5
2024-08-07 17:01:03.445 | INFO     | vlhf.hugging_face:download_dataset:116 - Adding image filename to dataset


Map:   0%|          | 0/1029 [00:00<?, ? examples/s]

2024-08-07 17:01:04.264 | INFO     | vlhf.hugging_face:download_dataset:121 - Adding bbox label name to dataset


Map:   0%|          | 0/1029 [00:00<?, ? examples/s]

Saving images:   0%|          | 0/1029 [00:00<?, ?it/s]

In [3]:
hf.dataset[0]['objects']

{'area': [3796, 1596, 152768, 81002],
 'bbox': [[302.0, 109.0, 73.0, 52.0],
  [810.0, 100.0, 57.0, 28.0],
  [160.0, 31.0, 248.0, 616.0],
  [741.0, 68.0, 202.0, 401.0]],
 'category': [4, 4, 0, 0],
 'category_name': ['mask', 'mask', 'coverall', 'coverall'],
 'id': [114, 115, 116, 117]}

In [4]:
hf.to_vl(vl_session=vl)

2024-08-07 17:01:21.902 | INFO     | vlhf.hugging_face:to_vl:177 - Preparing upload to Visual Layer
2024-08-07 17:01:29.724 | INFO     | vlhf.visual_layer:create_dataset:24 - Creating dataset: cppe-5


Dataset url: https://app.visual-layer.com/dataset/a330d2aa-549b-11ef-be8d-dacb63a3a95e


2024-08-07 17:01:51.277 | INFO     | vlhf.visual_layer:create_dataset:26 - Dataset cppe-5 successfully created in Visual Layer!


In [ ]:
! tar -xvf /home/dnth/Desktop/vl-hf-workflow/notebooks/saved_images/keremberke/csgo-object-detection.tar -C /home/dnth/Desktop/vl-hf-workflow/notebooks/saved_images/keremberke

In [ ]:
import pandas as pd

df = pd.read_parquet("saved_images/keremberke/object_annotations.parquet")

In [ ]:
df

In [ ]:
# plot the bounding boxes from the dataframe
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def plot_bounding_boxes_from_df(df):
    for filename, group in df.groupby('filename'):
        image = Image.open(f"saved_images/keremberke/{filename}")
        draw = ImageDraw.Draw(image)
        
        for i, row in group.iterrows():
            x, y, w, h = row['col_x'], row['row_y'], row['width'], row['height']
            label = row['label']
            draw.rectangle([x, y, x+w, y+h], outline="red", width=3)
            left, top, right, bottom = draw.textbbox((x, y-10), str(label))
            draw.rectangle((left-5, top-5, right+5, bottom+5), fill="red")
            draw.text((x, y-10), str(label), fill="white")

        
        plt.figure(figsize=(12, 8))
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Image: {filename}")
        plt.show()

In [ ]:
plot_bounding_boxes_from_df(df.sample(5))